In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd
from shapely.geometry import Point
import branca
 
# Load the cyclone data

excel_file_path = '/Users/monmita/Desktop/CycloneApplication/NI_9.xlsx'  # Updated path

df = pd.read_excel(excel_file_path)
 
# Load Gujarat boundary shapefile (ensure you have a shapefile for Gujarat's boundary)

gujarat_shapefile = '/Users/monmita/Desktop/CycloneApplication/Gujarat.shp'  # Updated path

gujarat_gdf = gpd.read_file(gujarat_shapefile)
 
# Reproject Gujarat boundary to the same CRS as the cyclone points (Web Mercator EPSG:3857)

gujarat_gdf = gujarat_gdf.to_crs(epsg=3857)
 
# Initialize the map centered on a specific location

map_center = [22.82, 77.68]  # Center around the Gulf of Mexico

zoom_start = 5
 
# Create the map with Esri World Imagery tiles

my_map = folium.Map(location=map_center, zoom_start=zoom_start, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr='Esri World Imagery')
 
# Add the Gujarat shapefile as a GeoJSON overlay on the map

folium.GeoJson(gujarat_gdf).add_to(my_map)
 
# Add a MarkerCluster to the map

marker_cluster = MarkerCluster().add_to(my_map)
 
# Initialize variables for landfall count

landfall_count = 0
 
# Create a list to store valid landfall points inside Gujarat and their buffers

landfall_points = []

buffers = []
 
# Create a color map for wind speed (from low to high)

colormap = branca.colormap.linear.RdYlBu_09.scale(df['Maximum Wind Speed'].min(), df['Maximum Wind Speed'].max())
 
# Iterate through the DataFrame to process each landfall

for index, row in df.iterrows():

    tc_number = row['TC Number']

    latitude = row['Latitude']

    longitude = row['Longitude']

    year = row['Year']

    wind_speed = row['Maximum Wind Speed']

    landfall = row['Landfall']  # Landfall indicator

    # If it's a landfall point (landfall == 1)

    if landfall == 1:

        # Create a Point geometry for the landfall

        landfall_point = Point(longitude, latitude)

        # Convert the Point to a GeoDataFrame to use the .to_crs() method

        gdf_point = gpd.GeoDataFrame(geometry=[landfall_point], crs="EPSG:4326")

        # Reproject the landfall point to EPSG:3857 (same as Gujarat)

        gdf_point = gdf_point.to_crs(epsg=3857)
 
        # Check if the reprojected landfall point is within the Gujarat boundary

        if gujarat_gdf.geometry.contains(gdf_point.geometry.iloc[0]).any():

            # Create a 5 km buffer around the landfall point

            buffer = gdf_point.geometry.buffer(5000)  # 5000 meters = 5 km

            # Assign a color to the buffer based on wind speed

            buffer_color = colormap(wind_speed)

            # Add landfall point to the map

            folium.CircleMarker(

                location=(latitude, longitude),

                radius=5,

                color="red",

                fill=True,

                fill_color="red",

                fill_opacity=0.7,

                popup=f'TC {tc_number}, Year {year}, Wind Speed {wind_speed}'

            ).add_to(marker_cluster)

            # Add the buffer to the map with color based on wind speed

            folium.GeoJson(buffer, style_function=lambda x, color=buffer_color: {

                'fillColor': color,

                'color': color,

                'weight': 1,

                'fillOpacity': 0.5

            }).add_to(my_map)
 
            # Append landfall point and buffer to the lists

            landfall_points.append(landfall_point)

            buffers.append(buffer)
 
            # Increment the landfall count

            landfall_count += 1
 
# Display the number of points inside Gujarat

print(f"Number of landfall points inside Gujarat: {landfall_count}")
 
# Add a marker with the count of landfall points inside Gujarat

folium.Marker(

    location=map_center,

    popup=f'Number of landfall points inside Gujarat: {landfall_count}',

    icon=folium.Icon(color='red')

).add_to(my_map)
 
# Add the color bar for wind speed to the map

colormap.caption = 'Wind Speed (km/h)'

colormap.add_to(my_map)
 
# Save the map to an HTML file

output_file_path = '/Users/monmita/Desktop/CycloneApplication/NI_9_with_spectrum_buffers.html'  # Updated path

my_map.save(output_file_path)
 
# Save the landfall points and buffers as a GeoDataFrame and shapefile

gdf_landfall_points = gpd.GeoDataFrame(geometry=landfall_points, crs="EPSG:4326")

gdf_buffers = gpd.GeoDataFrame(geometry=buffers, crs="EPSG:3857")  # Buffer is in EPSG:3857
 
# Reproject buffers back to EPSG:4326 for consistency with original landfall points

gdf_buffers = gdf_buffers.to_crs(epsg=4326)
 
# Save landfall points and buffers as a shapefile

shapefile_output_path = '/Users/monmita/Desktop/CycloneApplication/NI_9_landfall_with_spectrum_buffers.shp'  # Updated path

gdf_buffers.to_file(shapefile_output_path)
 
# Display the map in the Jupyter notebook

display(my_map)
 
# Plot the Gujarat shapefile and landfall points with buffers using Matplotlib

fig, ax = plt.subplots(figsize=(10, 10))

gujarat_gdf.plot(ax=ax, color='lightblue', edgecolor='black')
 
# Plot landfall points as red dots

for landfall_point in landfall_points:

    ax.plot(landfall_point.x, landfall_point.y, marker='o', color='red', markersize=5)
 
# Plot buffers with their corresponding colors

for i, buffer in enumerate(buffers):

    buffer.plot(ax=ax, facecolor=colormap(df['Maximum Wind Speed'].iloc[i]), edgecolor='blue', alpha=0.5)
 
ax.set_title("Landfall Points and Colored Buffers based on Wind Speed", fontsize=16)

plt.show()

 

Number of landfall points inside Gujarat: 457


GeometryTypeError: Unknown geometry type: 'featurecollection'